In [66]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sb3_contrib import RecurrentPPO
from stable_baselines3.common.evaluation import evaluate_policy
import stable_baselines3

import gymnasium as gym
import gym_trading_env
from gym_trading_env.utils.history import History
from gym_trading_env.downloader import download
import datetime

from stable_baselines3.common.env_checker import check_env


In [67]:
feature_list = []
for col in df.columns:
    feature_list.append(col)

In [68]:
def feature_cols(df):
    df["feature_close"] = df["close"]
    df["feature_high"] = df["high"]
    df["feature_low"] = df["low"]
    df["feature_open"] = df["open"]
    df["feature_vol"] = df["vol"]

def calculate_reward_cols(df, window_size):
    prices = df["feature_close"]
    # This needs to calculate only when the model has a position.
    log_returns = prices.pct_change().apply(lambda x: np.log(1 + x))
    df["log_return"] = log_returns
    df["avg_log_return"] = df["log_return"].rolling(window=window_size).mean()
    df["avg_log_return_std"] = df["avg_log_return"].std()


df = pd.read_csv("./data/indicators.csv", parse_dates=["date"], index_col="date")
feature_cols(df)
#calculate_reward_cols(df, 30)
df.dropna(inplace=True)

print(df.shape)

(1863, 45)


In [78]:
env = gym.make(
    "TradingEnv",
    name="BTCUSD",
    df=df,
    positions=[0, 1],
    )

obs = env.reset()
obs

(array([ 3.2752628e+00,  1.4165925e+01,  4.1008701e+00,  1.6234456e+00,
        -2.6530146e+01,  4.3732853e+01,  1.7506500e+02, -5.6983795e+01,
        -2.0178050e+02, -7.5750748e+01, -1.2202590e+02, -1.4951251e+02,
         4.6771442e+01,  6.7115637e+02,  6.6474899e+01,  1.4187622e+01,
        -1.3082519e+01, -2.9601984e+01, -2.9691364e+01,  2.0186232e+01,
        -4.2739563e+00,  5.0000000e+01,  4.5869595e+01,  3.6106823e+01,
         3.2149414e+01,  2.9272116e+01,  3.6292324e+01,  3.7901905e+01,
         1.1211000e+03,  1.3767250e+03,  1.6377167e+03,  1.6452625e+03,
         2.0441687e+03,  2.0290250e+03,  1.3752109e+03,  1.5100000e+04,
         1.5267800e+04,  1.4100000e+04,  1.4917000e+04,  2.1320000e+04,
         0.0000000e+00,  0.0000000e+00], dtype=float32),
 {'idx': 0,
  'step': 0,
  'date': numpy.datetime64('2018-01-04T00:00:00.000000000'),
  'position_index': 0,
  'position': 0,
  'real_position': 0,
  'data_low': 14100.0,
  'data_high': 15267.8,
  'data_open': 14917.0,
  'd

Running env with random action taken each step

In [70]:
done, truncated = False, False
observation, info = env.reset()
while not done and not truncated:
    # Pick a position by its index in your position list (=[-1, 0, 1])....usually something like : position_index = your_policy(observation)
    position_index = env.action_space.sample() # At every timestep, pick a random position index from your position list (=[-1, 0, 1])
    observation, reward, done, truncated, info = env.step(position_index)

Market Return : 44.32%   |   Portfolio Return : -40.79%   |   


In [71]:
model = RecurrentPPO('MlpLstmPolicy', # feed-forward neural network with multiple hidden layers
            env, # environment in which the agent interacts and learns
            verbose=1, # enables the training progress to be printed during the learning process
            gamma=0.95, # determines the importance of future rewards compared to immediate rewards
            n_steps=15, # steps to collect samples from the environment before performing an update
            ent_coef=0.01, # encourages exploration by adding entropy to the policy loss
            learning_rate=0.001, # controls the step size at which model's parameters are updated based on the gradient of the loss function
            clip_range=0.1, # limits the update to a certain range to prevent large policy updates
            batch_size=15)

model.learn(total_timesteps=100)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------
| time/              |     |
|    fps             | 168 |
|    iterations      | 1   |
|    time_elapsed    | 0   |
|    total_timesteps | 15  |
----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 61           |
|    iterations           | 2            |
|    time_elapsed         | 0            |
|    total_timesteps      | 30           |
| train/                  |              |
|    approx_kl            | 0.0043043974 |
|    clip_fraction        | 0.22         |
|    clip_range           | 0.1          |
|    entropy_loss         | -0.69        |
|    explained_variance   | -17.2        |
|    learning_rate        | 0.001        |
|    loss                 | -0.0367      |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0211      |
|    value_los

In [76]:
obs = env.reset()

for i in range(100):
    action, _state = model.predict(obs)
    obs, reward, done, info = env.step(action)



ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.